In [1]:
import pandas as pd
import pymongo
import json
import datetime as dt

In [2]:
# import all cvs from data folder
earthquake_csv = pd.read_csv("../raw/Earthquake_data.csv")
floods_csv = pd.read_csv("../floods_data.csv")
tornado_csv = pd.read_csv("../tor_output.csv")
hurricane_csv = pd.read_csv("../hurricane_cleaned_df.csv")

/Users/knytes/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
earthquake_df = pd.DataFrame(earthquake_csv)
floods_df = pd.DataFrame(floods_csv)
hurricane_df = pd.DataFrame(hurricane_csv)
tornado_df = pd.DataFrame(tornado_csv)

## EARTHQUAKE DATA CLEANING

In [4]:
earthquake_df["Date"] = earthquake_df.Year.map(str)+'-'+ earthquake_df.Month.map(str) + '-' + earthquake_df.Day.map(str)


In [5]:
earthquake_df.Date = pd.to_datetime(earthquake_df.Date)

In [6]:
earthquake_df = earthquake_df.fillna(0)

In [7]:
earthquake_df = earthquake_df.dropna()

In [8]:
earthquake_df.Hour = earthquake_df.Hour.astype(int)

In [9]:
earthquake_df.Minute = earthquake_df.Minute.astype(int)

In [10]:
earthquake_df.Minute = earthquake_df.Minute.map("{:02}".format)  
earthquake_df.Hour  = earthquake_df.Hour.map("{:02}".format)

In [11]:
earthquake_df["Time"] = earthquake_df.Hour.map(str) + ":" + earthquake_df.Minute

In [12]:
earthquake_df = earthquake_df.rename(columns = {"Latitude": "latitude", "Longitude": "longitude"})

In [13]:
earthquake_df.Time = pd.to_datetime(earthquake_df.Time).dt.time

In [14]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 19 columns):
Year                           194 non-null int64
Month                          194 non-null int64
Day                            194 non-null int64
Hour                           194 non-null object
Minute                         194 non-null object
Location                       194 non-null object
latitude                       194 non-null float64
longitude                      194 non-null float64
Magnitude                      194 non-null float64
Deaths                         194 non-null float64
Injuries                       194 non-null float64
Damage in Millions             194 non-null float64
Damage Scale                   194 non-null float64
Number of Houses Destroyed     194 non-null float64
Scale of houses destroyed      194 non-null float64
Number of Houses Damaged       194 non-null float64
Scale of houses  damaged       194 non-null float64
Date                

## HURRICANE DATA CLEANING

In [15]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 12 columns):
Unnamed: 0      57095 non-null int64
index           57095 non-null int64
season          57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.2+ MB


In [16]:
hurricane_df.head()

,Unnamed: 0,index,season,basin,subbasin,name,iso_time,nature,latitude,longitude,usa_wind,usa_pressure
0,0,1,2018,EP,MM,XAVIER,2018-11-06 06:00:00 UTC,NR,18.9371,-107.832,38.0,1005.0
1,1,2,2018,EP,CP,OLIVIA,2018-09-14 06:00:00 UTC,NR,18.9405,-164.639,29.0,1008.0
2,2,5,2018,EP,CP,OLIVIA,2018-09-14 12:00:00 UTC,NR,18.8559,-165.699,29.0,1008.0
3,3,11,1966,EP,CP,KATHY,1966-10-20 06:00:00 UTC,TS,44.6143,180.000,55.0,NaN
4,4,12,1992,EP,CP,DAN,1992-10-25 06:00:00 UTC,TS,11.6714,180.229,30.0,NaN


In [17]:
hurricane_df = hurricane_df.rename(columns = {"season": "Year"})

In [18]:
hurricane_df = hurricane_df.drop(columns = ["Unnamed: 0"])

In [19]:
hurricane_df[["Date","Time", "Time_Zone"]] = hurricane_df.iso_time.str.split(" ", expand = True)

In [20]:
hurricane_df.Date = pd.to_datetime(hurricane_df.Date)
hurricane_df.Time = pd.to_datetime(hurricane_df.Time).dt.time

## FLOOD DATA CLEANING

In [21]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 17 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null object
event_end_time      34586 non-null object
event_timezone      34586 non-null object
damage_property     34586 non-null float64
event_latitude      34586 non-null float64
event_longitude     34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
Start_Date          34586 non-null object
Start_Time          34586 non-null object
End_Date            34586 non-null object
End_Time            34586 non-null object
dtypes: float64(4), int64(2), object(11)
memory usage: 4.5+ MB


In [22]:
floods_df[["Date", "Time"]] = floods_df.event_begin_time.str.split(" ", expand = True)

In [23]:
floods_df[["Year", "Month", "Day"]] = floods_df.Start_Date.str.split("-", expand = True)

In [24]:
floods_df = floods_df.drop(columns = ["Month", "Day"])

In [25]:
floods_df.Date = pd.to_datetime(floods_df.Date)
floods_df.Time = pd.to_datetime(floods_df.Time).dt.time

In [26]:
floods_df = floods_df.rename(columns = {"event_latitude": "latitude", "event_longitude": "longitude"})


In [27]:
floods_df = floods_df.drop(columns = ["Start_Date", "End_Time", "End_Date","End_Date","Start_Time"])

In [28]:
floods_df.event_begin_time = pd.to_datetime(floods_df.event_begin_time)
floods_df.event_end_time = pd.to_datetime(floods_df.event_end_time)


In [29]:
floods_df.Year = floods_df.Year.astype(int)

## TORNADO DATA CLEANING

In [30]:
tornado_df.head()

,Date,Time,State,Fscale,Injured,Fatalities,Property_loss,SLAT,SLON,ELAT,ELON,Length,Width
0,1950-01-03,11:00:00,MO,3,3.0,0.0,6.0,38.77,-90.22,38.83,-90.03,15.2855,137.111517
1,1950-01-03,11:55:00,IL,3,3.0,0.0,5.0,39.10,-89.30,39.12,-89.23,5.7924,118.829982
2,1950-01-03,16:00:00,OH,1,1.0,0.0,4.0,40.88,-84.58,0.00,0.00,0.1609,9.140768
3,1950-01-13,05:25:00,AR,3,1.0,1.0,3.0,34.40,-94.37,0.00,0.00,0.9654,15.539305
4,1950-01-25,19:30:00,MO,2,5.0,0.0,5.0,37.60,-90.68,37.63,-90.65,3.7007,274.223035


In [31]:
tornado_df[["Year", "Month", "Day"]] = tornado_df.Date.str.split("-", expand = True)


In [32]:
tornado_df = tornado_df.drop(columns = ["Month", "Day"])

In [33]:
tornado_df.Date = pd.to_datetime(tornado_df.Date, errors = 'coerce')
# tornado_df.Time = pd.to_datetime(tornado_df.Time, errors = 'coerce')


In [34]:
tornado_df.Property_loss = tornado_df.Property_loss.replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)

In [35]:
tornado_df=tornado_df.dropna()

In [36]:
tornado_df.Year = tornado_df.Year.astype(int)

## Change all column to lower case

In [37]:
floods_df.columns = map(str.lower, floods_df.columns)
earthquake_df.columns = map(str.lower, earthquake_df.columns)
hurricane_df.columns = map(str.lower, hurricane_df.columns)
tornado_df.columns = map(str.lower, tornado_df.columns)

## View all DataFrames

In [38]:
floods_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34586 entries, 0 to 34585
Data columns (total 16 columns):
episode_id          34586 non-null int64
state               34586 non-null object
event_type          34586 non-null object
event_begin_time    34586 non-null datetime64[ns]
event_end_time      34586 non-null datetime64[ns]
event_timezone      34586 non-null object
damage_property     34586 non-null float64
latitude            34586 non-null float64
longitude           34586 non-null float64
cz_type             34586 non-null object
cz_fips_code        34586 non-null int64
damage_crops        34586 non-null float64
source              34586 non-null object
date                34586 non-null datetime64[ns]
time                34586 non-null object
year                34586 non-null int64
dtypes: datetime64[ns](3), float64(4), int64(3), object(6)
memory usage: 4.2+ MB


In [39]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 19 columns):
year                           194 non-null int64
month                          194 non-null int64
day                            194 non-null int64
hour                           194 non-null object
minute                         194 non-null object
location                       194 non-null object
latitude                       194 non-null float64
longitude                      194 non-null float64
magnitude                      194 non-null float64
deaths                         194 non-null float64
injuries                       194 non-null float64
damage in millions             194 non-null float64
damage scale                   194 non-null float64
number of houses destroyed     194 non-null float64
scale of houses destroyed      194 non-null float64
number of houses damaged       194 non-null float64
scale of houses  damaged       194 non-null float64
date                

In [40]:
hurricane_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57095 entries, 0 to 57094
Data columns (total 14 columns):
index           57095 non-null int64
year            57095 non-null int64
basin           57095 non-null object
subbasin        57095 non-null object
name            57095 non-null object
iso_time        57095 non-null object
nature          57095 non-null object
latitude        57095 non-null float64
longitude       57095 non-null float64
usa_wind        57095 non-null float64
usa_pressure    30937 non-null float64
date            57095 non-null datetime64[ns]
time            57095 non-null object
time_zone       57095 non-null object
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 6.1+ MB


In [41]:
tornado_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66136 entries, 0 to 66140
Data columns (total 14 columns):
date             66136 non-null datetime64[ns]
time             66136 non-null object
state            66136 non-null object
fscale           66136 non-null object
injured          66136 non-null float64
fatalities       66136 non-null float64
property_loss    66136 non-null int64
slat             66136 non-null float64
slon             66136 non-null float64
elat             66136 non-null float64
elon             66136 non-null float64
length           66136 non-null float64
width            66136 non-null float64
year             66136 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(2), object(3)
memory usage: 7.6+ MB


## Convert all the dataframes into json format

In [42]:
hurricane_data = json.loads(hurricane_df.T.to_json()).values()
floods_data = json.loads(floods_df.T.to_json()).values()
earthquake_data = json.loads(earthquake_df.T.to_json()).values()
tornado_data = json.loads(tornado_df.T.to_json()).values()

## NoSQL

In [43]:
# connect to mongod local server
conn = 'mongodb://localhost:27017'
# create client 
client = pymongo.MongoClient(conn)
# create database
# db.weather_data.drop()
db = client.weather_data


In [44]:
db.hurricane_data.drop()
db.hurricane_data.insert(hurricane_data)

/Users/knytes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1d4f024ab51bc0954ee5b6'),
 ObjectId('5e1d4f024ab51bc0954ee5b7'),
 ObjectId('5e1d4f024ab51bc0954ee5b8'),
 ObjectId('5e1d4f024ab51bc0954ee5b9'),
 ObjectId('5e1d4f024ab51bc0954ee5ba'),
 ObjectId('5e1d4f024ab51bc0954ee5bb'),
 ObjectId('5e1d4f024ab51bc0954ee5bc'),
 ObjectId('5e1d4f024ab51bc0954ee5bd'),
 ObjectId('5e1d4f024ab51bc0954ee5be'),
 ObjectId('5e1d4f024ab51bc0954ee5bf'),
 ObjectId('5e1d4f024ab51bc0954ee5c0'),
 ObjectId('5e1d4f024ab51bc0954ee5c1'),
 ObjectId('5e1d4f024ab51bc0954ee5c2'),
 ObjectId('5e1d4f024ab51bc0954ee5c3'),
 ObjectId('5e1d4f024ab51bc0954ee5c4'),
 ObjectId('5e1d4f024ab51bc0954ee5c5'),
 ObjectId('5e1d4f024ab51bc0954ee5c6'),
 ObjectId('5e1d4f024ab51bc0954ee5c7'),
 ObjectId('5e1d4f024ab51bc0954ee5c8'),
 ObjectId('5e1d4f024ab51bc0954ee5c9'),
 ObjectId('5e1d4f024ab51bc0954ee5ca'),
 ObjectId('5e1d4f024ab51bc0954ee5cb'),
 ObjectId('5e1d4f024ab51bc0954ee5cc'),
 ObjectId('5e1d4f024ab51bc0954ee5cd'),
 ObjectId('5e1d4f024ab51bc0954ee5ce'),
 ObjectId('5e1d4f024ab51b

In [45]:
db.flood_data.drop()
db.flood_data.insert(floods_data)

/Users/knytes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1d4f034ab51bc0954fc4bd'),
 ObjectId('5e1d4f034ab51bc0954fc4be'),
 ObjectId('5e1d4f034ab51bc0954fc4bf'),
 ObjectId('5e1d4f034ab51bc0954fc4c0'),
 ObjectId('5e1d4f034ab51bc0954fc4c1'),
 ObjectId('5e1d4f034ab51bc0954fc4c2'),
 ObjectId('5e1d4f034ab51bc0954fc4c3'),
 ObjectId('5e1d4f034ab51bc0954fc4c4'),
 ObjectId('5e1d4f034ab51bc0954fc4c5'),
 ObjectId('5e1d4f034ab51bc0954fc4c6'),
 ObjectId('5e1d4f034ab51bc0954fc4c7'),
 ObjectId('5e1d4f034ab51bc0954fc4c8'),
 ObjectId('5e1d4f034ab51bc0954fc4c9'),
 ObjectId('5e1d4f034ab51bc0954fc4ca'),
 ObjectId('5e1d4f034ab51bc0954fc4cb'),
 ObjectId('5e1d4f034ab51bc0954fc4cc'),
 ObjectId('5e1d4f034ab51bc0954fc4cd'),
 ObjectId('5e1d4f034ab51bc0954fc4ce'),
 ObjectId('5e1d4f034ab51bc0954fc4cf'),
 ObjectId('5e1d4f034ab51bc0954fc4d0'),
 ObjectId('5e1d4f034ab51bc0954fc4d1'),
 ObjectId('5e1d4f034ab51bc0954fc4d2'),
 ObjectId('5e1d4f034ab51bc0954fc4d3'),
 ObjectId('5e1d4f034ab51bc0954fc4d4'),
 ObjectId('5e1d4f034ab51bc0954fc4d5'),
 ObjectId('5e1d4f034ab51b

In [46]:
db.earthquake_data.drop()
db.earthquake_data.insert(earthquake_data)

/Users/knytes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1d4f044ab51bc095504bd7'),
 ObjectId('5e1d4f044ab51bc095504bd8'),
 ObjectId('5e1d4f044ab51bc095504bd9'),
 ObjectId('5e1d4f044ab51bc095504bda'),
 ObjectId('5e1d4f044ab51bc095504bdb'),
 ObjectId('5e1d4f044ab51bc095504bdc'),
 ObjectId('5e1d4f044ab51bc095504bdd'),
 ObjectId('5e1d4f044ab51bc095504bde'),
 ObjectId('5e1d4f044ab51bc095504bdf'),
 ObjectId('5e1d4f044ab51bc095504be0'),
 ObjectId('5e1d4f044ab51bc095504be1'),
 ObjectId('5e1d4f044ab51bc095504be2'),
 ObjectId('5e1d4f044ab51bc095504be3'),
 ObjectId('5e1d4f044ab51bc095504be4'),
 ObjectId('5e1d4f044ab51bc095504be5'),
 ObjectId('5e1d4f044ab51bc095504be6'),
 ObjectId('5e1d4f044ab51bc095504be7'),
 ObjectId('5e1d4f044ab51bc095504be8'),
 ObjectId('5e1d4f044ab51bc095504be9'),
 ObjectId('5e1d4f044ab51bc095504bea'),
 ObjectId('5e1d4f044ab51bc095504beb'),
 ObjectId('5e1d4f044ab51bc095504bec'),
 ObjectId('5e1d4f044ab51bc095504bed'),
 ObjectId('5e1d4f044ab51bc095504bee'),
 ObjectId('5e1d4f044ab51bc095504bef'),
 ObjectId('5e1d4f044ab51b

In [47]:
db.tornado_data.drop()
db.tornado_data.insert(tornado_data)

/Users/knytes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5e1d4f044ab51bc095504c99'),
 ObjectId('5e1d4f044ab51bc095504c9a'),
 ObjectId('5e1d4f044ab51bc095504c9b'),
 ObjectId('5e1d4f044ab51bc095504c9c'),
 ObjectId('5e1d4f044ab51bc095504c9d'),
 ObjectId('5e1d4f044ab51bc095504c9e'),
 ObjectId('5e1d4f044ab51bc095504c9f'),
 ObjectId('5e1d4f044ab51bc095504ca0'),
 ObjectId('5e1d4f044ab51bc095504ca1'),
 ObjectId('5e1d4f044ab51bc095504ca2'),
 ObjectId('5e1d4f044ab51bc095504ca3'),
 ObjectId('5e1d4f044ab51bc095504ca4'),
 ObjectId('5e1d4f044ab51bc095504ca5'),
 ObjectId('5e1d4f044ab51bc095504ca6'),
 ObjectId('5e1d4f044ab51bc095504ca7'),
 ObjectId('5e1d4f044ab51bc095504ca8'),
 ObjectId('5e1d4f044ab51bc095504ca9'),
 ObjectId('5e1d4f044ab51bc095504caa'),
 ObjectId('5e1d4f044ab51bc095504cab'),
 ObjectId('5e1d4f044ab51bc095504cac'),
 ObjectId('5e1d4f044ab51bc095504cad'),
 ObjectId('5e1d4f044ab51bc095504cae'),
 ObjectId('5e1d4f044ab51bc095504caf'),
 ObjectId('5e1d4f044ab51bc095504cb0'),
 ObjectId('5e1d4f044ab51bc095504cb1'),
 ObjectId('5e1d4f044ab51b